. Fusion et Traitement des Données Démographiques et Géographiques pour Analyse

In [ ]:
import pandas as pd

# Définir la fonction pour traiter les données
def store_data(demographic_data, geographic_data):
    # Fusionner les données sur la colonne 'Location'
    merged_data = pd.merge(demographic_data, geographic_data, on='Location')
    # Afficher les données fusionnées
    return merged_data

# Lire les fichiers CSV
demographic_data = pd.read_csv('demographic_data.csv')
geographic_data = pd.read_csv('geographic_data.csv')

# Appeler la fonction avec les deux DataFrames
store_data(demographic_data, geographic_data)


Automatisation de la Récupération des Données de Pollution Atmosphérique à Partir de Coordonnées Géographiques

In [ ]:
import requests
import json
import pandas as pd
import os

# Lire les fichiers CSV
demographic_data = pd.read_csv('demographic_data.csv')
geographic_data = pd.read_csv('geographic_data.csv')

# ajout manuel des colonnes Latitude et Longitude dans geographic_data.csv
# (Idéalement, ces données devraient être obtenues d'une source fiable ou ajoutées manuellement)

# Fusionner les deux jeux de données sur la colonne 'Location' pour obtenir toutes les informations en un seul DataFrame
merged_data = pd.merge(demographic_data, geographic_data, on='Location')

# Créer un dictionnaire de coordonnées à partir des données fusionnées
coordinates = {}
for index, row in merged_data.iterrows():
    city = row['Location']
    lat = row['Latitude']
    lon = row['Longitude']
    coordinates[city] = (lat, lon)

# Récupérer la clé API depuis les variables d'environnement
API_KEY = os.getenv('OPENWEATHER_API_KEY')
BASE_URL = 'http://api.openweathermap.org/data/2.5/air_pollution'

def get_air_pollution_data(lat, lon):
    url = f"{BASE_URL}?lat={lat}&lon={lon}&appid={API_KEY}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}")
        return None

# Récupération des données de pollution pour chaque ville
for city, (lat, lon) in coordinates.items():
    print(f"Récupération des données pour {city}...")
    data = get_air_pollution_data(lat, lon)
    print(json.dumps(data, indent=2))


Enrichissement des Données Géographiques avec les Indices AQI et Composants de Pollution Atmosphérique

In [ ]:
import requests
import json
import pandas as pd
from datetime import datetime

# Supposons que 'Latitude' et 'Longitude' sont les noms corrects des colonnes
def get_pollution_aqi_data(row, API_KEY):
    lat = row['Latitude']
    lon = row['Longitude']
    url = f'http://api.openweathermap.org/data/2.5/air_pollution?lat={lat}&lon={lon}&appid={API_KEY}'
    response = requests.get(url)
    data = response.json()
    return data['list'][0]['main']['aqi']

def get_pollution_components_data(row, API_KEY):
    lat = row['Latitude']
    lon = row['Longitude']
    url = f'http://api.openweathermap.org/data/2.5/air_pollution?lat={lat}&lon={lon}&appid={API_KEY}'
    response = requests.get(url)
    data = response.json()
    return data['list'][0]['components']

# Ajout des données de pollution de l'air dans le DataFrame
merged_data['AQI Pollution'] = merged_data.apply(lambda row: get_pollution_aqi_data(row, API_KEY), axis=1)

# Récupérer les composants de la pollution
components_df = merged_data.apply(lambda row: pd.Series(get_pollution_components_data(row, API_KEY)), axis=1)

# Joindre les composants au DataFrame principal
merged_data = merged_data.join(components_df)

# Supprimer les colonnes lat/lon car elles ne sont plus nécessaires
merged_data.drop(columns=['Latitude', 'Longitude'], axis=1, inplace=True)

# Ajouter la date courante
merged_data['date'] = datetime.now().date()

# Définir 'Location' comme index
merged_data.set_index('Location', inplace=True)

# Afficher les 5 premières lignes du DataFrame
print(merged_data.head(5))


## Chargement des Données dans un Fichier

In [ ]:
# Enregistrement du rapport
merged_data.to_csv('air_pollution_data.csv')

## Documentation des étapes et la justification des choix

# Documenter les Étapes :

Lecture des Fichiers CSV :
Importer les données sources nécessaires à l'analyse.

Fusion des Données :
Intégrer les informations démographiques et géographiques dans un seul DataFrame pour faciliter l'analyse croisée.

Création du Dictionnaire de Coordonnées :
Structurer les coordonnées géographiques pour une utilisation simple et rapide lors des appels API.

Récupération des Données API :
Enrichir les données existantes avec des informations actuelles sur la pollution de l'air, ce qui est essentiel pour l'analyse environnementale.

Nettoyage des Données :
Supprimer les colonnes inutiles et ajouter des informations de suivi (comme la date) pour garder un historique des données.

# Justification des Choix :

Fusion des Données :
La fusion des données permet une meilleure cohérence et facilite l'accès à toutes les informations nécessaires à l'analyse en un seul endroit.

Utilisation d'une API :
L'intégration de données externes via une API permet d'obtenir des informations en temps réel sur la pollution de l'air, ajoutant ainsi de la pertinence et de l'actualité aux données existantes.

Nettoyage et Structuration :
Le nettoyage des données assure la clarté et la précision des résultats, tandis que la structuration du DataFrame final facilite son utilisation dans des analyses ou visualisations ultérieures.